# Create an InterActive Graph of NTRL Lab Analysis Results

In [6]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict
from datetime import date
import datetime
import re
from datetime import timedelta
import openpyxl
from sklearn.metrics import mean_squared_error
import plotly.express as px

In [7]:
#used in the analysis results data. cleans up the sampling time entry
def time_cleaner(x):
    time_regex = re.compile(r'[0-8a-zA-Z:]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    timeact = time_regex.findall(x)
    newtime = ''.join(timeact)
    t = parse(x)
    new_str_time = str(t.hour)+':'+str(t.minute)+':'+str(t.second)
    return new_str_time
def ave_filler(dfname, colname):
    dfcn = dfname[colname]
    try:        
        dfcn = pd.to_numeric(dfcn, errors='coerce')
    except:
        print('error found in' + str(colname))
        pass
    try:
        dfname[colname] = dfcn.where(dfcn.notnull(), other=(dfcn.fillna(method='ffill') + dfcn.fillna(method='bfill'))/2)
    except:
        print('nan not fixed' + str(colname))   
    return dfname
def generate_df_from_analysis(workbookTab, col_sampleDate, col_sampleTime, col_analysis, col_title):
    #get the start and end sample dates
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
    #gets the date day from the B column of the analysis file if there is value in the cell
    for i in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
            Smpl_day_index.append(i)
    #get the ending row that has date day
    for j in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
            smpl_end_index = j
    for y in range(0, len(Smpl_day_index)):
        if Smpl_day_index[y] != Smpl_day_index[-1]:      
            for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                if workbookTab[str(col_analysis)+str(x)].value is None: 
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

        if Smpl_day_index[y] == Smpl_day_index[-1]:
            for x in range(Smpl_day_index[y], smpl_end_index+1):
                if workbookTab[str(col_analysis)+str(x)].value is None:   
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

    Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
    Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
    Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
    Chem_Element_df.set_index('DATETIME', inplace=True)

    return Chem_Element_df

def generate_add_trace(figname, df_elemental, axis_bool):
    colname = df_elemental.columns[0]
    figname.add_trace(go.Scatter(x=df_elemental.index, y=df_elemental[colname], name=colname), secondary_y=axis_bool,)
#---------------------------------------

In [8]:
class LabResults():
    def __init__(self, monthyear, filePath):
        #____monthyear format examle is datetime.date(2020,7,1)
        #____filepath example is '\\thgobna001\userdata\THPAL\...\ANALYSIS RESULTS 2020\7) July _2020.xlsx'
        #____add r' at the start of the filepath to apply the regex that allows the use of the filepath as raw
        #____sef.workbook is the uploaded excel workbook of the entire analysis files so it takes time to load
        #____Create an instance of the workbook by giving it a varible name with the sample format: Jan_2020
        
        import openpyxl
        import io
        with open(filePath, "rb") as f:
            in_mem_file = io.BytesIO(f.read())
        
        self.workbook = openpyxl.load_workbook(in_mem_file, data_only=True)
        self.monthyear = monthyear   
        
    def generate_df_from_analysis(self, tabname, list_lab_index):
        #____tabname is a string type name of the tab in the laboratory analysis excel file.
        #____-------Example tabnames: '104PU01', '105TK03', '106TK01'
        #____-------Inside these tabs are the specific analysis results done on the sample taken from the 'tabname' area .
        #____list_lab_index is a list with the sample format is: ['B', 'C', 'E', '104_Pb']
        #____-------the first letter 'B' is the column letter where the day of the month is written as integers 1,2,3..30
        #____-------the second letter 'C' is the column letter where the sampling time is written; e.g. '7:00:00 AM'
        #____-------the third letter 'E' is the column where the results of the specific analysis are listed
        #____-------the last item in the list is the string you'll use as the new title; e.g. '104_Pb'
        
        workbookTab = self.workbook[tabname]
        self.list_lab_index = list_lab_index
        col_sampleDate = self.list_lab_index[0]
        col_sampleTime = self.list_lab_index[1]
        col_analysis = self.list_lab_index[2]
        col_title = self.list_lab_index[3]
        currentmonthyr = self.monthyear
        
        #get the start and end sample dates
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
        #gets the date day from the B column of the analysis file if there is value in the cell
        for i in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
                Smpl_day_index.append(i)
        #get the ending row that has date day
        for j in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
                smpl_end_index = j
        for y in range(0, len(Smpl_day_index)):
            if Smpl_day_index[y] != Smpl_day_index[-1]:      
                for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                    if workbookTab[str(col_analysis)+str(x)].value is None: 
                        pass
                    else:
                        try:
                            if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                                date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                                +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                                Chem_Element_datelist.append(parse(date))  
                                chem_element = workbookTab[str(col_analysis)+str(x)].value
                                Chem_Element_list.append(chem_element)
                            else:
                                date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                                +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                                Chem_Element_datelist.append(parse(date))  
                                chem_element = workbookTab[str(col_analysis)+str(x)].value
                                Chem_Element_list.append(chem_element)
                        except:
                            pass

            if Smpl_day_index[y] == Smpl_day_index[-1]:
                for x in range(Smpl_day_index[y], smpl_end_index+1):
                    if workbookTab[str(col_analysis)+str(x)].value is None:   
                        pass
                    else:
                        try:
                            if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                                date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                                +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                                Chem_Element_datelist.append(parse(date))
                                chem_element = workbookTab[str(col_analysis)+str(x)].value
                                Chem_Element_list.append(chem_element)
                            else:
                                date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                                +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                                Chem_Element_datelist.append(parse(date))
                                chem_element = workbookTab[str(col_analysis)+str(x)].value
                                Chem_Element_list.append(chem_element)
                        except:
                            pass

        Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
        Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
        Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
        Chem_Element_df.set_index('DATETIME', inplace=True)

        return Chem_Element_df

In [9]:
def join_DF(dfsList1):
    df = dfsList1[0]
    for i in range(1,len(dfsList1)):
        df = df.join(dfsList1[i], how='outer')
    return df

def append_monthsDF(dfsListmonthly): #takes the list of dataframes from monthly lab results of one sampling area
    df = dfsListmonthly[0]
    for i in range(1,len(dfsListmonthly)):
        df = df.append(dfsListmonthly[i], ignore_index=True)
    return df

In [10]:
#define the filepaths and the monthyr data for year 2019

currentmonthyr_01_2017 = datetime.date(2017,1,1)
currentmonthyr_02_2017 = datetime.date(2017,2,1)
currentmonthyr_03_2017 = datetime.date(2017,3,1)
currentmonthyr_04_2017 = datetime.date(2017,4,1)
currentmonthyr_05_2017 = datetime.date(2017,5,1)
currentmonthyr_06_2017 = datetime.date(2017,6,1)
currentmonthyr_07_2017 = datetime.date(2017,7,1)
currentmonthyr_08_2017 = datetime.date(2017,8,1)
currentmonthyr_09_2017 = datetime.date(2017,9,1)
currentmonthyr_10_2017 = datetime.date(2017,10,1)
currentmonthyr_11_2017 = datetime.date(2017,11,1)
currentmonthyr_12_2017 = datetime.date(2017,12,1)

currentmonthyr_01_2018 = datetime.date(2018,1,1)
currentmonthyr_02_2018 = datetime.date(2018,2,1)
currentmonthyr_03_2018 = datetime.date(2018,3,1)
currentmonthyr_04_2018 = datetime.date(2018,4,1)
currentmonthyr_05_2018 = datetime.date(2018,5,1)
currentmonthyr_06_2018 = datetime.date(2018,6,1)
currentmonthyr_07_2018 = datetime.date(2018,7,1)
currentmonthyr_08_2018 = datetime.date(2018,8,1)
currentmonthyr_09_2018 = datetime.date(2018,9,1)
currentmonthyr_10_2018 = datetime.date(2018,10,1)
currentmonthyr_11_2018 = datetime.date(2018,11,1)
currentmonthyr_12_2018 = datetime.date(2018,12,1)

currentmonthyr_01_2019 = datetime.date(2019,1,1)
currentmonthyr_02_2019 = datetime.date(2019,2,1)
currentmonthyr_03_2019 = datetime.date(2019,3,1)
currentmonthyr_04_2019 = datetime.date(2019,4,1)
currentmonthyr_05_2019 = datetime.date(2019,5,1)
currentmonthyr_06_2019 = datetime.date(2019,6,1)
currentmonthyr_07_2019 = datetime.date(2019,7,1)
currentmonthyr_08_2019 = datetime.date(2019,8,1)
currentmonthyr_09_2019 = datetime.date(2019,9,1)
currentmonthyr_10_2019 = datetime.date(2019,10,1)
currentmonthyr_11_2019 = datetime.date(2019,11,1)
currentmonthyr_12_2019 = datetime.date(2019,12,1)

currentmonthyr_01_2020 = datetime.date(2020,1,1)
currentmonthyr_02_2020 = datetime.date(2020,2,1)
currentmonthyr_03_2020 = datetime.date(2020,3,1)
currentmonthyr_04_2020 = datetime.date(2020,4,1)
currentmonthyr_05_2020 = datetime.date(2020,5,1)
currentmonthyr_06_2020 = datetime.date(2020,6,1)
currentmonthyr_07_2020 = datetime.date(2020,7,1)
currentmonthyr_08_2020 = datetime.date(2020,8,1)
currentmonthyr_09_2020 = datetime.date(2020,9,1)


#filepath

filepath_01_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\01) January_2017.xlsx'
filepath_02_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\02) February_2017.xlsx'
filepath_03_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\03) March_2017 .xlsx'
filepath_04_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\04) April_2017.xlsx'
filepath_05_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\05) May_2017.xlsx'
filepath_06_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\06) June 2017.xlsx'
filepath_07_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\07) July 2017.xlsx'
filepath_08_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\08) August 2017.xlsx'
filepath_09_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\09) September 2017.xlsx'
filepath_10_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\10) October 2017.xlsx'
filepath_11_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\11) November 2017.xlsx'
filepath_12_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\12) December 2017.xlsx'

filepath_01_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\1.) January 2018.xlsx'
filepath_02_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\2.) February 2018.xlsx'
filepath_03_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\3.) March 2018.xlsx'
filepath_04_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\4.) April 2018.xlsx'
filepath_05_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\5.) May 2018.xlsx'
filepath_06_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\6.) June 2018.xlsx'
filepath_07_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\7.) July 2018.xlsx'
filepath_08_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\8.) August 2018.xlsx'
filepath_09_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\9.) September 2018.xlsx'
filepath_10_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\10.) October 2018.xlsx'
filepath_11_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\11.) November 2018.xlsx'
filepath_12_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\12.) December 2018.xlsx'

filepath_01_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\1.) January 2019.xlsx'
filepath_02_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\2.) February 2019.xlsx'
filepath_03_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\3.) March 2019.xlsx'
filepath_04_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\4.) April  2019.xlsx'
filepath_05_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\5.) May _2019.xlsx'
filepath_06_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\6.) June  _2019.xlsx'
filepath_07_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\7.) July_2019.xlsx'
filepath_08_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\8.) August_2019.xlsx'
filepath_09_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\9.) September_2019.xlsx'
filepath_10_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\10.) October_ 2019.xlsx'
filepath_11_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\11.) November_ 2019.xlsx'
filepath_12_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\12.) December_ 2019.xlsx'

filepath_01_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\01 JANUARY 2020\1) JANUARY_2020.xlsx'
filepath_02_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\02 FEBRUARY 2020\2) FEBRUARY _2020.xlsx'
filepath_03_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\03 MARCH 2020\3) March _2020.xlsx'
filepath_04_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\04 APRIL 2020\4) April _2020.xlsx'
filepath_05_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\05 MAY 2020\5) MAY_2020.xlsx'
filepath_06_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\06 JUNE 2020\6) June _2020.xlsx'
filepath_07_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\07 JULY 2020\7) July _2020.xlsx'
filepath_08_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\08 AUGUST 2020\8) August _2020.xlsx'
filepath_09_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\9) September _2020.xlsx'




dateList20172020 = [currentmonthyr_01_2017,currentmonthyr_02_2017,currentmonthyr_03_2017,currentmonthyr_04_2017,currentmonthyr_05_2017,currentmonthyr_06_2017,
                    currentmonthyr_07_2017,currentmonthyr_08_2017,currentmonthyr_09_2017,currentmonthyr_10_2017,currentmonthyr_11_2017,currentmonthyr_12_2017,
                    currentmonthyr_01_2018,currentmonthyr_02_2018,currentmonthyr_03_2018,currentmonthyr_04_2018,currentmonthyr_05_2018,currentmonthyr_06_2018,
                    currentmonthyr_07_2018,currentmonthyr_08_2018,currentmonthyr_09_2018,currentmonthyr_10_2018,currentmonthyr_11_2018,currentmonthyr_12_2018,
                    currentmonthyr_01_2019,currentmonthyr_02_2019,currentmonthyr_03_2019,currentmonthyr_04_2019,currentmonthyr_05_2019,currentmonthyr_06_2019,
                    currentmonthyr_07_2019,currentmonthyr_08_2019,currentmonthyr_09_2019,currentmonthyr_10_2019,currentmonthyr_11_2019,currentmonthyr_12_2019,
                    currentmonthyr_01_2020,currentmonthyr_02_2020,currentmonthyr_03_2020,currentmonthyr_04_2020,currentmonthyr_05_2020,currentmonthyr_06_2020,
                    currentmonthyr_07_2020,currentmonthyr_08_2020,currentmonthyr_09_2020]


filepathList20172020 = [filepath_01_2017,filepath_02_2017,filepath_03_2017,filepath_04_2017,filepath_05_2017,filepath_06_2017,
                        filepath_07_2017,filepath_08_2017,filepath_09_2017,filepath_10_2017,filepath_11_2017,filepath_12_2017,
                        filepath_01_2018,filepath_02_2018,filepath_03_2018,filepath_04_2018,filepath_05_2018,filepath_06_2018,
                        filepath_07_2018,filepath_08_2018,filepath_09_2018,filepath_10_2018,filepath_11_2018,filepath_12_2018,
                        filepath_01_2019,filepath_02_2019,filepath_03_2019,filepath_04_2019,filepath_05_2019,filepath_06_2019,
                        filepath_07_2019,filepath_08_2019,filepath_09_2019,filepath_10_2019,filepath_11_2019,filepath_12_2019,
                        filepath_01_2020,filepath_02_2020,filepath_03_2020,filepath_04_2020,filepath_05_2020,filepath_06_2020,
                        filepath_07_2020,filepath_08_2020,filepath_09_2020]

In [11]:
print(len(dateList20172020[:17]))
print(len(filepathList20172020[:17]))
print((filepathList20172020[16]))

17
17
\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\5.) May 2018.xlsx


In [12]:
print((filepathList20172020[17]))

\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\6.) June 2018.xlsx


In [16]:
dateList20172020[6]

datetime.date(2017, 7, 1)

In [13]:
# List corresponding to the cell location of each specific analysis type per sample source
# For 2017 to 2018_05 only. The cell location for pH60 and filtration time

tk05_Pb = ['B', 'C', 'E', 'tk05_Pb']
tk05_Zn = ['B', 'C', 'F', 'tk05_Zn']
tk05_Cu = ['B', 'C', 'G', 'tk05_Cu']
tk05_Ni = ['B', 'C', 'H', 'tk05_Ni']
tk05_Co = ['B', 'C', 'I', 'tk05_Co']
tk05_Fe = ['B', 'C', 'J', 'tk05_Fe']
tk05_Mn = ['B', 'C', 'K', 'tk05_Mn']
tk05_Cr = ['B', 'C', 'L', 'tk05_Cr']
tk05_Ca = ['B', 'C', 'M', 'tk05_Ca']
tk05_Si = ['B', 'C', 'N', 'tk05_Si']
tk05_Al = ['B', 'C', 'O', 'tk05_Al']
tk05_Mg = ['B', 'C', 'P', 'tk05_Mg']

for i in range(0,len(dateList20172020)):
    
    monthLab = LabResults(dateList20172020[i], filepathList20172020[i])
    df_month = monthLab.generate_df_from_analysis('105TK05', tk05_Pb).join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Zn), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Cu), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Ni), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Co), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Fe), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Mn), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Cr), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Ca), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Si), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Al), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Mg), how='outer')
        
    df_month.to_excel(r'C:\Users\v.t.flores\Documents\ZnDrop_Proj_DataGather_Monthly_105TK05{}.xlsx'.format(str(dateList20172020[i])))


In [18]:
# List corresponding to the cell location of each specific analysis type per sample source
# For 2017 to 2018_05 only. The cell location for pH60 and filtration time

MStk01_ZnDropRate_1 = ['B', 'C', 'BC', 'Zn_drop_act']


for i in range(0,7):
    
    monthLab = LabResults(dateList20172020[i], filepathList20172020[i])
    df_month = monthLab.generate_df_from_analysis('106TK01', MStk01_ZnDropRate_1)
        
    df_month.to_excel(r'C:\Users\v.t.flores\Documents\ZnDrop_Proj_DataGather_Monthly_106TK01ZnDrop{}.xlsx'.format(str(dateList20172020[i])))


In [19]:
# List corresponding to the cell location of each specific analysis type per sample source
# For 2017 to 2018_05 only. The cell location for pH60 and filtration time

MStk01_ZnDropRate_2 = ['B', 'C', 'BD', 'Zn_drop_act']

for i in range(7,len(dateList20172020)):
    
    monthLab = LabResults(dateList20172020[i], filepathList20172020[i])
    df_month = monthLab.generate_df_from_analysis('106TK01', MStk01_ZnDropRate_2)
        
    df_month.to_excel(r'C:\Users\v.t.flores\Documents\ZnDrop_Proj_DataGather_Monthly_106TK01ZnDrop{}.xlsx'.format(str(dateList20172020[i])))